# Validation

## Task


Исходные данные - Yandex Cup 2022 RecSys:

* Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
* Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
* Описание данных - в архиве likes_data.zip три файла:
    * train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
    * test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
    * track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
* Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
* Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

<b>Итоговая задача</b> - построить схему валидации для данного соревнования с учетом особенностей сорвенования

* Между train и test не должно быть общих пользователей
* Количество фолдов задается через параметр класса n_folds
* В test должно быть не более p последних треков (параметр класса p)

## Imports

In [1]:
import os

import numpy as np
import pandas as pd
import requests
from sklearn.model_selection import KFold

## Data

In [2]:
DATASET_PATH="../data/ya_cup"

In [3]:
!mkdir -p $DATASET_PATH

�訡�� � ᨭ⠪�� �������.


In [4]:
# Скачать датасет через Y.Disk API
DATASET_PATH = os.getenv('DATASET_PATH', '../data/ya_cup')

disk_api_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download'
likes_url = 'https://disk.yandex.ru/d/SI1aAooPn9i8TA'
download_link = requests.get(f'{disk_api_url}?public_key={likes_url}').json()['href']

with open(f'{DATASET_PATH}/data.zip', 'wb') as f:
    response = requests.get(download_link)
    f.write(response.content)

In [24]:
!unzip $DATASET_PATH/data.zip -d $DATASET_PATH > /dev/null
!unzip $DATASET_PATH/likes/likes_data.zip -d $DATASET_PATH > /dev/null
!rm -rf $DATASET_PATH/{likes,data.zip,__MACOSX}

���⥬� �� 㤠���� ���� 㪠����� ����.
���⥬� �� 㤠���� ���� 㪠����� ����.


In [6]:
with open( f"{DATASET_PATH}/train", "r") as train:
    lines = train.readlines()

In [7]:
likes = []

for i, line in enumerate(lines):
    track_ids = list(map(int, line.split()))
    user_likes = np.zeros((len(track_ids), 2), dtype=int)
    user_likes[:, 0] = i
    user_likes[:, 1] = track_ids[::-1]
    likes.append(user_likes)


In [8]:
df = pd.DataFrame(np.vstack(likes), columns=['user_id', 'track_id'])
del likes
del lines

df.user_id = df.user_id.astype(np.uint32)
df.track_id = df.track_id.astype(np.uint32)

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94188634 entries, 0 to 94188633
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   uint32
 1   track_id  uint32
 2   order     uint16
dtypes: uint16(1), uint32(2)
memory usage: 898.3 MB


,user_id,track_id,order
0,0,388242,53
1,0,278503,52
2,0,102795,51
3,0,470957,50
4,0,159637,49


## Solution

In [22]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed

    def split(self, df):
        users = df['user_id'].unique()
        order = df.groupby('user_id').cumcount()
        # df.order
        user_kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)
        for train_users, test_users in user_kfold.split(users):
            train_mask = df['user_id'].isin(train_users)
            test_mask = df['user_id'].isin(test_users) & (order < self.p)
            yield train_mask, test_mask

In [23]:
n_folds = 3  # Количество фолдов задается через параметр класса n_folds
p = 2

cv = UsersKFoldPOut(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train = df[train_mask]
    test = df[test_mask]
    assert len(set(train['user_id'].unique()).intersection(test['user_id'].unique())) == 0, 'Между train и test не должно быть общих пользователей'
    assert test.groupby('user_id').count().values.max() <= p, 'В test должно быть не более `p` последних треков'

    test_first_user = test.iloc[0]['user_id']
    test_fold_user_tracks = test[test['user_id'] == test_first_user]['track_id'].values
    all_user_tracks = df[df['user_id'] == test_first_user]['track_id'].values
    assert np.all(test_fold_user_tracks == all_user_tracks[:p]), 'Неверный track_ids юзера в `test`'
